<a href="https://colab.research.google.com/github/jlembury/kickflip-cartography-demo/blob/main/kickflip_cartography_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="margin-left: 0; margin-right: auto;">
    <tr>
        <td><img src="https://github.com/jlembury/kickflip-cartography-demo/blob/main/img/img-nada-kJvjDlqTkR0-unsplash.jpg?raw=1"></td>
        <td><h1 style="margin-bottom: 0;">Kickflip Cartography: </h1>
            <h3 style="margin-top: 0;">Finding the Perfect Location for a Skate Park</h3><br><br>
            <h4 style="margin-bottom: 0;">GIS Awareness Week 2025, hosted by AIRC x Ecologik</h4>
            <h4 style="margin-top: 0; margin-bottom: 0;">Jessica Embury <em>(she/her/hers)</em></h4>
            <h4 style="margin-top: 0; margin-bottom: 0;">Email: <a href="mailto:jessica@gisjess.com">jessica@gisjess.com</a></h4>
        </td>
    </tr>
</table>
<hr>
<br>

### Project Overview
A local community group wants to build a skate park somewhere in the City of San Diego. The group wants to create a safe hangout for skateboarders <em>and</em> maximize access to skate parks throughout the city. To meet these objectives, the site for the future skate park must:
1. be located in a <b>San Diego public park</b> at least 1 mile from an <b>existing skate park</b>, and
2. be located in a <b>zip code</b> with demonstrated need, per <b>2025 San Diego police call records</b>.
<br>
<hr>
<br>

### Data
- <em>parks</em>: San Diego Regional Data Warehouse, https://tinyurl.com/sd-data-warehouse
- <em>skateParks</em>: The City of San Diego, https://www.sandiego.gov/park-and-recreation/centers/skateparks
- <em>zipcodes</em>: San Diego Regional Data Warehouse, https://tinyurl.com/sd-data-warehouse
- <em>policeCalls</em>: Data SD, https://data.sandiego.gov/datasets/police-calls-for-service/
<br>
<hr>
<br>

### Spatial Operations
<img src="https://github.com/jlembury/kickflip-cartography-demo/blob/main/img/img-spatial-operations.jpg?raw=1">
<hr>

###Objective 1
Identify San Diego parks that are more than 1 mile from a skate park.

1. <u>Geocode</u> the street addresses in <em>skateParks</em> → <em>skateParkPoints</em>
2. Use a <u>buffer</u> operation to create 1-mile zones around the points in <em>skateParkPoints</em>  → <em>skateParkBuffers</em>
3. Use a <u>difference</u> operation to remove <em>parks</em> that overlap with <em>skateParkBuffers</em> → <em>parkCandidates</em>

In [6]:
!pip install geopandas

import pandas as pd
import geopandas as gpd
import time
from geopy.geocoders import Nominatim
from shapely.geometry import Point

skateParksFile = "https://raw.githubusercontent.com/jlembury/kickflip-cartography-demo/refs/heads/main/data/skate-park-locations.csv"

In [18]:
skateParks = pd.read_csv(skateParksFile)
print(skateParks.head(2))

geolocator = Nominatim(user_agent="kickflip-cartography-demo")

geometry = []
for i,row in skateParks.iterrows():
  location = geolocator.geocode(skateParks.at[i, "address"])
  geometry.append(Point(location.longitude, location.latitude))
  time.sleep(1)

skateParkPoints = gpd.GeoDataFrame(skateParks, geometry=geometry, crs="EPSG:4326")
print(skateParkPoints.head(2))
skateParkPoints.to_file("skateParkPoints.shp")




                     skate_park_name  \
0  Bill and Maxine Wilson Skate Park   
1           Carmel Valley Skate Park   

                                     address  
0         702 S 30th St, San Diego, CA 92113  
1  12600 El Camino Real, San Diego, CA 92130  


                     skate_park_name  \
0  Bill and Maxine Wilson Skate Park   
1           Carmel Valley Skate Park   

                                     address                     geometry  
0         702 S 30th St, San Diego, CA 92113  POINT (-117.12959 32.69841)  
1  12600 El Camino Real, San Diego, CA 92130   POINT (-117.23779 32.9296)  


/tmp/ipython-input-471360418.py:14: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  skateParkPoints.to_file("skateParkPoints.shp")
/usr/local/lib/python3.12/dist-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/usr/local/lib/python3.12/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'skate_park_name' to 'skate_park'
  ogr_write(


In [28]:
skateParkPoints.to_crs("EPSG:3857", inplace=True)
skateParkBuffers = skateParkPoints[["skate_park_name", "address"]]
skateParkBuffers["geometry"] = skateParkPoints.geometry.buffer(1610)
skateParkBuffers

,skate_park_name,address,geometry
0,Bill and Maxine Wilson Skate Park,"702 S 30th St, San Diego, CA 92113","POLYGON ((-13037196.739 3855340.433, -13037204..."
1,Carmel Valley Skate Park,"12600 El Camino Real, San Diego, CA 92130","POLYGON ((-13049241.474 3885962.986, -13049249..."
2,Central Avenue Skate Park,"3680 Central Ave, San Diego, CA 92105","POLYGON ((-13034827.381 3861574.562, -13034835..."
3,Cesar Solis Skate Park,"4895 Del Sol Blvd, San Diego, CA 92154","POLYGON ((-13025722.916 3838613.682, -13025730..."
4,Charles L. Lewis Memorial Skate Park,"6610 Potomac St, San Diego, CA 92139","POLYGON ((-13028628.945 3853824.621, -13028636..."
5,Chicano Park Skate Park,"1949 Logan Ave, San Diego, CA 92113","POLYGON ((-13038831.165 3855729.854, -13038838..."
6,Del Sur Skate Park,"15816 Paseo Montenero, San Diego, CA 92127","POLYGON ((-13038426.173 3897467.424, -13038433..."
7,Linda Vista Skateboard Park,"7064 Levant St, San Diego, CA 92111","POLYGON ((-13041608.753 3867719.254, -13041616..."
8,Pacific Highlands Skate Park,"5977 Village Center Loop Road, San Diego, CA 9...","POLYGON ((-13043548.106 3890179.071, -13043555..."
9,Park de La Cruz Skate Park,"3901 Landis St, San Diego, CA 92105","POLYGON ((-13035119.873 3861521.965, -13035127..."


###Objective 2
Find park(s) located in the zipcode with the most skateboarding police calls.

5. <u>Geocode</u> the skateboarding incident locations from <em>policeCalls</em> → <em>policeCallPoints</em>
6. Use a <u>within</u> operation to count the number of incidents per zipcode in <em>zipcodes</em> → <em>zipcodeCounts</em>
7. Use a <u>within</u> operation to find park(s) (<em>parkCandidates</em>) in the zipcode(s) (<em>zipcodeCounts</em>) with the most skateboarding incidents → <em>parkFinalists</em>